# Set Up (imports including psspy)

In [42]:
!pip install pandas
!pip install numpy

In [ ]:
import pandas as pd
import numpy as np
import os, sys
print(sys.version) # should be 3.11.9

In [2]:
print(sys.path)

['C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv', '', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\Pythonwin']


In [3]:
PSSE_LOCATION = r"C:\Program Files\PTI\PSSE36\36.1\PSSPY311" # this is where the psspy module is located on my machine
sys.path.append(PSSE_LOCATION)
os.environ['PATH'] += ';' + PSSE_LOCATION

In [4]:
import psse3601
import psspy

# Import Case Data

In [ ]:
# initialize case
psspy.psseinit(39)


 PSS(R)E Xplore Version 36
 Copyright (c) 1976-2025
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyright
 laws  of  non-U.S.  countries  and  by  application  of  international
 treaties.  All  Rights  Reserved  Under  The  Copyright  Laws.


           SIEMENS POWER TECHNOLOGIES INTERNATIONAL

    50 BUS POWER SYSTEM SIMULATOR--PSS(R)E Xplore-36.1.0

             INITIATED ON SAT, APR 26 2025  16:27


0

In [11]:
# read case
psspy.read(0, "IEEE 39 bus.RAW")


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 39 records for Bus data
 Starting processing of Load data records
 Finished processing of 31 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 2 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 10 records for Generator data
 Starting processing of Branch data records
 Finished processing of 34 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 48 records for 12 Transformer data
 Starting processing of Area data records
 Finished processing of 1 records for Area data
 Starting processing of 

0

In [12]:
# run load flow analysis
psspy.fnsl([0,0,0,1,1,0,99,0]) # flat start newton-raphson load flow


 Ordering network...
 Diagonals = 38   Off-diagonals = 66   Maximum size = 90

  ITER       DELTAP      BUS         DELTAQ      BUS        DELTA/V/      BUS       DELTAANG      BUS
   0         0.0007(      2     )    0.0022(      5     )
                                                             0.00001(     17     )   0.00000(     22     )
   1         0.0000(      2     )    0.0000(      5     )
                                                             0.00000(     18     )   0.00000(     25     )
   2         0.0000(      6     )    0.0000(     22     )


 Reached tolerance in 2 iterations

 Largest mismatch:     -0.00 MW      0.00 Mvar      0.00 MVA at bus 5 [5           1.0000]
 System total absolute mismatch:                    0.00 MVA

 SWING BUS SUMMARY:
   BUS#-SCT X-- NAME --X BASKV      PGEN     PMAX    PMIN      QGEN     QMAX    QMIN
     31     31          1.0000     571.3   9999.9     0.0     363.9    800.0  -500.0


0

In [36]:
from itertools import groupby
from operator import itemgetter
attr_type = itemgetter(0)

def subsystem_info(name, attributes, sid=-1, inservice=True):
    """
    Returns requested attributes from the PSS(r)E subsystem API
    for the given subsystem id and subsystem element name.
    e.g. to retrieve bus attributes "NAME", "NUMBER" and "PU"
      
      subsystem_info('bus', ["NAME", "NUMBER", "PU"])
    where the 'bus' `name` argument comes from the original
    PSS(r)E subsystem API naming convention found in Chapter 8 of the
    PSS(r)E API.
    abusint  # bus
    amachint # mach
    aloadint # load
    Args:
      inservice [optional]: True (default) to list only information
         for in service elements;
      sid [optional]: list only information for elements in this
         subsystem id (-1, all elements by default).  
    
    """
    name = name.lower()
    gettypes = getattr(psspy, 'a%stypes' % name)
    apilookup = {
            'I': getattr(psspy, 'a%sint' % name),
            'R': getattr(psspy, 'a%sreal' % name),
            'X': getattr(psspy, 'a%scplx' % name),
            'C': getattr(psspy, 'a%schar' % name), }

    result = []
    ierr, attr_types = gettypes(attributes)

    for k, group in groupby(list(zip(attr_types, attributes)), key=attr_type):
        func = apilookup[k]
        strings = list(list(zip(*group))[1])
        ierr, res = func(sid, flag=1 if inservice else 2, string=strings)
        result.extend(res)

    return list(zip(*result))

In [39]:
cols = ['NUMBER', 'PU', 'SHUNTACT']
businfo = subsystem_info('bus', cols, sid=-1)
print(businfo)

[(1, 1.0517840385437012, -0j), (2, 1.0598753690719604, -0j), (3, 1.0565475225448608, -0j), (4, 1.0591864585876465, -112.1875991821289j), (5, 1.0735946893692017, -230.52110290527344j), (6, 1.074280858039856, -0j), (7, 1.0611553192138672, -0j), (8, 1.0585850477218628, -0j), (9, 1.054019570350647, -0j), (10, 1.0610601902008057, -0j), (11, 1.06428861618042, -0j), (12, 1.0493440628051758, -0j), (13, 1.0586985349655151, -0j), (14, 1.056139588356018, -0j), (15, 1.0408930778503418, -0j), (16, 1.0485306978225708, -0j), (17, 1.0517454147338867, -0j), (18, 1.0523276329040527, -0j), (19, 1.0559942722320557, -0j), (20, 0.9942230582237244, -0j), (21, 1.0435523986816406, -0j), (22, 1.0561085939407349, -0j), (23, 1.0513238906860352, -0j), (24, 1.0525844097137451, -0j), (25, 1.0658155679702759, -0j), (26, 1.062293291091919, -0j), (27, 1.0517330169677734, -0j), (28, 1.0554094314575195, -0j), (29, 1.0535695552825928, -0j), (30, 1.0475000143051147, -0j), (31, 0.9819999933242798, -0j), (32, 0.9830999970436

In [20]:
ierr, busIDs = psspy.abusint(sid=-1, string="ID")
ierr, busNums = psspy.abusint(sid=-1, flag=4, string="NUMBER")
print(busIDs)

None


In [ ]:
print(busNums)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 39]]


In [18]:
ierr, busNames = psspy.abuschar(sid=-1, string="NAME")
print(busNames)

[['1           ', '2           ', '3           ', '4           ', '5           ', '6           ', '7           ', '8           ', '9           ', '10          ', '11          ', '12          ', '13          ', '14          ', '15          ', '16          ', '17          ', '18          ', '19          ', '20          ', '21          ', '22          ', '23          ', '24          ', '25          ', '26          ', '27          ', '28          ', '29          ', '30          ', '31          ', '32          ', '33          ', '34          ', '35          ', '36          ', '37          ', '38          ', '39          ']]
